<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/RAG_SQTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages setting up

In [1]:
%pip install langchain \
langchain_community \
langchain_core \
langchain_google_genai \
python-dotenv \
pypdf

In [2]:
%pip install faiss-cpu

In [3]:
%pip install scikit-learn \
numpy

In [4]:
pip install -U sentence-transformers

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from sentence_transformers import SentenceTransformer
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
VietTransformer = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True)
#embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
pdf_files = ['/content/sample_data/KinhTevPhatTrien2024.pdf',
             '/content/sample_data/KinhTevPhatTrien2025.pdf',
             '/content/sample_data/KinhTevPhatTrien2025_t2.pdf']  # Adjust paths

In [6]:
print(pdf_files)

['/content/sample_data/KinhTevPhatTrien2024.pdf', '/content/sample_data/KinhTevPhatTrien2025.pdf', '/content/sample_data/KinhTevPhatTrien2025_t2.pdf']


#PDF-Preprocessing & VectorDB

In [7]:
documents = []
for pdf in pdf_files:
    pdf_loader = PyPDFLoader(pdf)
    documents.extend(pdf_loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Ensure embeddings are generated correctly
#embeddings = embedding_model.embed_documents([doc.page_content for doc in chunks])

# Pass embedded vectors to FAISS

with open("extracted_content.txt", "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"Chunk {i+1}:\n{chunk.page_content}\n\n{'='*50}\n\n")

print("📄 Extracted content saved to extracted_content.txt")

📄 Extracted content saved to extracted_content.txt


In [23]:
from sentence_transformers import SentenceTransformer

class CustomEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)  # ✅ Add trust_remote_code=True

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True)

    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True)[0]
    def __call__(self, text):
        """Make the class callable, so FAISS can use it."""
        return self.embed_query(text)

# Initialize the embedding model
embedding_model = CustomEmbedding("dangvantuan/vietnamese-document-embedding")

# Generate embeddings
doc_embeddings = embedding_model.embed_documents(["Xin chào!", "Hà Nội là thủ đô của Việt Nam."])
query_embedding = embedding_model.embed_query("Thành phố nào là thủ đô?")

print(doc_embeddings.shape, query_embedding.shape)


(2, 768) (768,)


In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

# Create the FAISS vector store
vector_db = FAISS.from_documents(chunks, embedding_model)


In [28]:
print(f"✅ Processed {len(chunks)} text chunks into FAISS vector database.")


✅ Processed 3010 text chunks into FAISS vector database.


In [29]:
query = "độ tuổi GenZ"
retrieved_docs = vector_db.similarity_search(query, k = 4)


In [30]:
for i, doc in enumerate(retrieved_docs[:4]):  # Show top 3
    print(f"\n📄 Document {i+1}:\n{doc.page_content}")


📄 Document 1:
Số 330 tháng 12/2024 66
1. Giới thiệu
Thế hệ Z thường được các nhà nghiên cứu xác định là sinh ra trong khoảng thời gian từ 1995 – 2012 
(Barhate & Dirani, 2022; Maloni & cộng sự, 2019). Với việc được đào tạo trình độ đại học, nhóm này đã 
gia nhập thị trường lao động được khoảng 7 năm và đang dần trở thành lực lượng lao động chính, đặc biệt là 
trong lĩnh vực kinh doanh và quản lý. Trong bối cảnh chuyển đổi lực lượng lao động như vậy, rất cần thiết

📄 Document 2:
2.2. Thế hệ Z
Là thế hệ mới nhất tham gia vào lực lượng lao động, thế hệ Z cho thấy sự khác biệt đáng kể trong hành 
vi và thái độ đối với công việc so với những thế hệ trước đây. Việc có thời gian đi học dài, có sự bao trùm 
của công nghệ và thiết bị di động, được sống trong một xã hội phát triển hơn đã tạo ra một thế hệ Z thiếu 
kinh nghiệm làm việc thực tế, coi trọng sự đa dạng và công bằng, dễ dàng rơi vào trạng thái lo âu và trầm

📄 Document 3:
sử dụng lại dịch vụ, giúp doanh nghệp có thể nâng cao doanh th

#LLM model

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    google_api_key=os.environ["GOOGLE_API_KEY"],
    model="gemini-2.0-flash-thinking-exp-01-21",
    temperature=0.7
)
print("✅ Chat model loaded successfully.")

✅ Chat model loaded successfully.


In [38]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

#Augment_Prompt

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

def augment_prompt(query):
    # Get top 3 results from the knowledge base
    results = vector_db.similarity_search(query, k=4)

    # Extract text, sources, and pages
    source_map = {}
    for doc in results:
        source = doc.metadata.get("source", "Unknown")
        page = doc.metadata.get("page", "Unknown")
        source_map[doc.page_content] = (source, page)

    # Construct the augmented prompt
    source_knowledge = "\n".join(source_map.keys())

    augmented_prompt = f"""Bạn là tư vấn viên của trường Sĩ Quan Thông Tin. Dựa vào nội dung tài liệu, hãy trả lời câu hỏi một cách chính xác và thân thiện bằng tiếng Việt.
    Không thêm thông tin ngoài nội dung tài liệu. Nếu không tìm thấy câu trả lời trong tài liệu, chỉ cần nói rằng bạn không biết.

    Nội dung tài liệu:
    {source_knowledge}

    Câu hỏi:
    {query}"""


    return augmented_prompt, source_map



#Answering Query

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# User question
question = "Độ tuổi Genz"

# Generate augmented prompt and retrieve sources
context, source_map = augment_prompt(question)

# Create human message for Gemini model
prompt = HumanMessage(content=context)

# Invoke Gemini model
res = chat_model.invoke([prompt])
response_text = res.content

# Get embeddings for LLM response
response_embedding = embedding_model.embed_query(response_text)

# Track relevant sources
relevant_sources = set()

for text, (source, page) in source_map.items():
    chunk_embedding = embedding_model.embed_query(text)  # Get embedding for each chunk
    similarity_score = cosine_similarity([response_embedding], [chunk_embedding])[0][0]

    if similarity_score >= 0.7:  # Threshold for relevance
        relevant_sources.add(f"{source} (Page {page+1})")

formatted_response = f"Response: {response_text}\nSources: {list(relevant_sources) if relevant_sources else ['No sources matched']}"
print(formatted_response)

#Generate Questions and Save to CSV

In [ ]:
'''import time
import csv
import os

csv_filename = "generated_questions.csv"

# Open CSV file for writing with UTF-8 BOM
with open(csv_filename, mode="a", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)

    # Write the header (only "Question" column now)
    writer.writerow(["Question"])

    for i, doc in enumerate(pdf_files):
        time.sleep(2)  # Avoid hitting API rate limits

        # Extract document name (without path)
        doc_name = os.path.basename(doc)

        # Select relevant chunks
        doc_chunks = chunks[i * 3 : (i + 1) * 3]
        doc_text = "\n".join([chunk.page_content.strip() for chunk in doc_chunks]).strip()

        if not doc_text:
            continue  # Skip if the document has no text

        # Generate questions with improved prompt
        prompt = HumanMessage(content=f"""Dựa vào nội dung tài liệu "{doc_name}", hãy tạo 5 câu hỏi đa dạng bằng tiếng Việt.
        Không đánh số thứ tự, không để lại khoảng trắng dư thừa, và mỗi câu hỏi phải có đủ ngữ cảnh để hiểu được tài liệu liên quan.""")

        response = chat_model.invoke([prompt])
        questions = [q.strip() for q in response.content.strip().split("\n") if q.strip()]  # Clean up questions

        # Write each question to the CSV file
        for question in questions:
            writer.writerow([question])

print(f"✅ Questions saved to {csv_filename}")
'''

#Generate Answers

In [ ]:
'''import pandas as pd
import time
from sklearn.metrics.pairwise import cosine_similarity

answer_csv = "generated_QA.csv"

# Load generated questions
df_questions = pd.read_csv("generated_questions.csv")
questions = df_questions["Question"].tolist()

answers = []
answer_sources = []

for question in questions:
    time.sleep(2)  # Avoid hitting API rate limits

    # Retrieve relevant document content
    context, _ = augment_prompt(question)
    prompt = HumanMessage(content=context)

    # Get answer from chatbot model
    answer_res = chat_model.invoke([prompt])
    answer = answer_res.content.strip()
    answers.append(answer)

    # Identify relevant sources for the answer
    relevant_sources = set()
    response_embedding = embedding_model.embed_query(answer)  # Embed answer

    for text, (source, page) in source_map.items():
        chunk_embedding = embedding_model.embed_query(text)  # Embed document chunk
        similarity_score = cosine_similarity([response_embedding], [chunk_embedding])[0][0]

        if similarity_score >= 0.7:  # Threshold for relevance
            relevant_sources.add(f"{source} (Page {page+1})")

    # Store relevant sources for answer
    source_list = "; ".join(relevant_sources) if relevant_sources else "No sources matched"
    answer_sources.append(source_list)

# Save answers and sources to CSV
df_answers = pd.DataFrame({"Answer": answers, "Relevant Source (Answer)": answer_sources})
df_answers.to_csv(answer_csv, index=False, encoding="utf-8-sig")

print(f"✅ Answers saved to {answer_csv}")
'''

#Reranker

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Load PhoRanker model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("itdainb/PhoRanker")



In [32]:
pip install --upgrade Pillow


In [33]:
model = AutoModelForSequenceClassification.from_pretrained("itdainb/PhoRanker")

In [34]:
import torch

def rerank_documents(query, docs):
    scored_docs = []
    for doc in docs:
        candidate_text = doc.page_content
        # Prepare the input pair (query, candidate)
        inputs = tokenizer(query, candidate_text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Get the score (assuming a single output neuron for relevance)
        score = outputs.logits[0][0].item()  # Access the first element directly
        scored_docs.append((doc, score))

    # Sort the documents by score (highest first)
    scored_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    # Return only the document objects in sorted order
    return [doc for doc, score in scored_docs]

In [35]:
query = "GenZ sinh ra trong khoảng thời gian nào?"
# Retrieve candidates (increase k to get more candidates for reranking)
retrieved_docs = vector_db.similarity_search(query, k=20)

# Rerank the documents using PhoRanker
reranked_docs = rerank_documents(query, retrieved_docs)

# Now you can use the top reranked documents (e.g., top 3)
top_docs = reranked_docs[:4]

# Print out the top documents for inspection
for i, doc in enumerate(retrieved_docs):
    print(f"\n📄 Document {i+1}:\n{doc.page_content}")

print("-------------------------------------------------------------------------")
for i, doc in enumerate(top_docs):
    print(f"\n📄 Document {i+1}:\n{doc.page_content}")



📄 Document 1:
Số 330 tháng 12/2024 66
1. Giới thiệu
Thế hệ Z thường được các nhà nghiên cứu xác định là sinh ra trong khoảng thời gian từ 1995 – 2012 
(Barhate & Dirani, 2022; Maloni & cộng sự, 2019). Với việc được đào tạo trình độ đại học, nhóm này đã 
gia nhập thị trường lao động được khoảng 7 năm và đang dần trở thành lực lượng lao động chính, đặc biệt là 
trong lĩnh vực kinh doanh và quản lý. Trong bối cảnh chuyển đổi lực lượng lao động như vậy, rất cần thiết

📄 Document 2:
2.2. Thế hệ Z
Là thế hệ mới nhất tham gia vào lực lượng lao động, thế hệ Z cho thấy sự khác biệt đáng kể trong hành 
vi và thái độ đối với công việc so với những thế hệ trước đây. Việc có thời gian đi học dài, có sự bao trùm 
của công nghệ và thiết bị di động, được sống trong một xã hội phát triển hơn đã tạo ra một thế hệ Z thiếu 
kinh nghiệm làm việc thực tế, coi trọng sự đa dạng và công bằng, dễ dàng rơi vào trạng thái lo âu và trầm

📄 Document 3:
sử dụng lại dịch vụ, giúp doanh nghệp có thể nâng cao doanh th

In [5]:
def augment_prompt_with_reranker(query):
    # Retrieve a larger set of candidate docs
    candidates = vector_db.similarity_search(query, k=15)
    # Rerank them using PhoRanker
    reranked_candidates = rerank_documents(query, candidates)

    # Chọn top 4 tài liệu sau khi rerank
    selected_docs = reranked_candidates[:4]

    # Tạo source map và trích dẫn nguồn
    source_map = {}
    formatted_sources = []

    for idx, doc in enumerate(selected_docs, start=1):
        source = doc.metadata.get("source", "Unknown")
        page = doc.metadata.get("page", "Unknown")
        source_map[doc.page_content] = (source, page)
        formatted_sources.append(f"[{idx}] {source} - Trang {page}")

    # Nội dung tài liệu dùng để trả lời
    source_knowledge = "\n\n".join(f"({i+1}) {doc.page_content}" for i, doc in enumerate(selected_docs))
    citation_info = "\n".join(formatted_sources)

    # Prompt tối ưu
    augmented_prompt = f"""Bạn là tư vấn viên của trường VGU. Hãy trả lời câu hỏi một cách chính xác, thân thiện, và trích dẫn nguồn gốc thông tin.
Bạn chỉ được sử dụng thông tin từ tài liệu dưới đây, không thêm nội dung không có trong tài liệu. Nếu không tìm thấy câu trả lời, chỉ cần nói rằng bạn không biết.

📌 **Nội dung tài liệu trích xuất**:
{source_knowledge}

❓ **Câu hỏi**:
{query}

📖 **Nguồn tài liệu**:
{citation_info}
"""

    print(candidates)
    print("--------------------------------------------------------------------------------------------")
    return augmented_prompt, source_map


In [ ]:
from sentence_transformers import CrossEncoder  # ✅ NEW
import numpy as np

# Load Cross-Encoder model
cross_encoder = CrossEncoder("itdainb/PhoRanker")

In [4]:
  # ✅ Use a ranking model

# User question
question = "độ tuổi GenZ"

# Retrieve chunks & sources
context, source_map = augment_prompt_with_reranker(question)

# Invoke LLM
prompt = HumanMessage(content=context)
res = chat_model.invoke([prompt])
response_text = res.content  # LLM-generated response
'''
# Track relevant sources
relevant_sources = set()

# Score each retrieved chunk using Cross-Encoder
scores = {}
for text, (source, page) in source_map.items():
    score = cross_encoder.predict([(question, text)])[0]  # ✅ Cross-Encoder scoring
    scores[(source, page)] = score

# Sort & filter sources by relevance threshold
sorted_sources = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for (source, page), score in sorted_sources:
    if score >= 0.5:  # ✅ Adjust threshold as needed
        relevant_sources.add(f"{source} (Page {page+1})")

# Format response
formatted_response = f"Response: {response_text}\nSources: {list(relevant_sources) if relevant_sources else ['No sources matched']}"
print(formatted_response)'''
print(response_text)


NameError: name 'vector_db' is not defined